# Analysis on the mental health disorders in the US

Student name: Jenny Nguyen

Student number: s223867709

Email address: s223867709@deakin.edu.au

Unit: SIT731 

## Introduction

Depression is a growing mental health concern that significantly impacts individuals' well-being and societal productivity. Understanding its prevalence across demographic factors, this report presents an analysis of depression before and after the COVID-19 pandemic, with a focus on understanding the distribution of depression over socio-economics factors such as age, gender, income, education level, and marital status.

Utilizing data from the National Health and Nutrition Examination Survey (NHANES), the study examines two distinct periods: pre-pandemic (2017–March 2020) and post-pandemic (August 2021–August 2023). By analyzing depression score, which is computed from severity of depression symptoms, we can categorize different depression levels accordingly for the analysis. This report aims to uncover how demographic and socio-economic factors influence the prevalence of moderate to severe depression across these timeframes. The findings aim to provide insights into the mental health challenges exacerbated by the pandemic and study into other contributing factors to support targeted interventions for affected groups.


## Report

### Load necessary libraries and data sets

In [69]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
from bokeh.models import HoverTool
from bokeh.plotting import figure, show
from bokeh.palettes import Spectral5
from bokeh.palettes import Spectral4
from bokeh.palettes import Vibrant3 as colors
from bokeh.io import output_notebook
from itertools import combinations
from bokeh.models import ColumnDataSource, FixedTicker
from bokeh.transform import linear_cmap

In [70]:
FILE_PATH = "C:/Users/phuon/nhanes_datasets/Depression/"
files ={
"bmx" : "BMX_L",
"p_bmx" : "P_BMX",
"sleep" : "SLQ_L",
"p_sleep" : "P_SLQ",
"income" : "INQ_L",
"p_income" : "P_INQ",
"depression" : "DPQ_L",
"p_depression" : "P_DPQ",
"demo" : "DEMO_L",
"p_demo" : "P_DEMO"}
dataframes = {}

for name, filename in files.items():
    print(name)
    full_path = f"{FILE_PATH}{filename}.XPT"
    dataframes[name] = pd.read_sas(full_path)
    print(dataframes[name].head())

bmx
       SEQN  BMDSTATS  BMXWT  BMIWT  BMXRECUM  BMIRECUM  BMXHEAD  BMIHEAD  \
0  130378.0       1.0   86.9    NaN       NaN       NaN      NaN      NaN   
1  130379.0       1.0  101.8    NaN       NaN       NaN      NaN      NaN   
2  130380.0       1.0   69.4    NaN       NaN       NaN      NaN      NaN   
3  130381.0       1.0   34.3    NaN       NaN       NaN      NaN      NaN   
4  130382.0       3.0   13.6    NaN       NaN       1.0      NaN      NaN   

   BMXHT  BMIHT  ...  BMXLEG  BMILEG  BMXARML  BMIARML  BMXARMC  BMIARMC  \
0  179.5    NaN  ...    42.8     NaN     42.0      NaN     35.7      NaN   
1  174.2    NaN  ...    38.5     NaN     38.7      NaN     33.7      NaN   
2  152.9    NaN  ...    38.5     NaN     35.5      NaN     36.3      NaN   
3  120.1    NaN  ...     NaN     NaN     25.4      NaN     23.4      NaN   
4    NaN    1.0  ...     NaN     NaN      NaN      1.0      NaN      1.0   

   BMXWAIST  BMIWAIST  BMXHIP  BMIHIP  
0      98.3       NaN   102.9     Na

In the above code chunk, XPT files downloaded from the NHANES dataset have been read and stored into a dictionary for easier access. By printing the first 5 rows of each dataset, we can see that all 10 datasets have been successfully read.

These datasets have been chosen to analyze factors that could affect depression level. According to US National Institute of Mental Health (NIMH), some major causes of depression include: Environmental factors like trauma, stress, or abuse; Psychological factors such as low self-esteem or negative thought patterns; Chronic illnesses and certain medications.

Understanding the main sources, some contributing factors can be drawn to investigate how they affect the depression level of different age groups, genders and marital status groups. Stress or negative thoughts could be the effect of low self-esteem in terms of low education level or poverty. On the other hand, illness or health deterioration can be resulted from lack of sleep or lack of physical activities. As a result, features chosen for the analysis include: education level, poverty level, sleep hours and BMI (representing physical health). List and source of the chosen datasets are as follows:
* Demographics Data: Demographic Variables and Sample Weights - include Gender, Age, Marital Status, Education Level
* Examination Data: Body Measures - include Body Mass Index (BMI)
* Questionaire Data:
   - Mental Health - Depression Screener - include Depression Indicators
   - Sleep Disorders - include Sleep hours on weekdays and weekends
   - Income - include Family monthly poverty level index

In [71]:
bmx_cols = ['Respondent sequence number','Body Measures Component Status Code','Weight (kg)','Weight Comment','Recumbent Length (cm)','Recumbent Length Comment','Head Circumference (cm)','Head Circumference Comment','Standing Height (cm)','Standing Height Comment','Body Mass Index (kg/m**2)','BMI Category - Children/Youth','Upper Leg Length (cm)','Upper Leg Length Comment','Upper Arm Length (cm)','Upper Arm Length Comment','Arm Circumference (cm)','Arm Circumference Comment','Waist Circumference (cm)','Waist Circumference Comment','Hip Circumference (cm)','Hip Circumference Comment']
p_bmx_cols = ['Respondent sequence number','Body Measures Component Status Code','Weight (kg)','Weight Comment','Recumbent Length (cm)','Recumbent Length Comment','Head Circumference (cm)','Head Circumference Comment','Standing Height (cm)','Standing Height Comment','Body Mass Index (kg/m**2)','BMI Category - Children/Youth','Upper Leg Length (cm)','Upper Leg Length Comment','Upper Arm Length (cm)','Upper Arm Length Comment','Arm Circumference (cm)','Arm Circumference Comment','Waist Circumference (cm)','Waist Circumference Comment','Hip Circumference (cm)','Hip Circumference Comment']
sleep_cols=['Respondent sequence number','Usual sleep time on weekdays or workdays','Usual wake time on weekdays or workdays','Sleep hours - weekdays or workdays','Usual sleep time on weekends','Usual wake time on weekends','Sleep hours - weekends']
p_sleep_cols=['Respondent sequence number','Usual sleep time on weekdays or workdays','Usual wake time on weekdays or workdays','Sleep hours - weekdays or workdays','Usual sleep time on weekends','Usual wake time on weekends','Sleep hours - weekends','How often do you snore?','How often do you snort or stop breathing','Ever told doctor had trouble sleeping?','How often feel overly sleepy during day?']
income_cols =['Respondent sequence number','Family monthly poverty level index','Family monthly poverty level category','Family has savings more than $20,000','Total savings/cash assets for the family']
p_income_cols =['Respondent sequence number','Family monthly poverty level index','Family monthly poverty level category']
depression_cols=['Respondent sequence number','Have little interest in doing things','Feeling down, depressed, or hopeless','Trouble sleeping or sleeping too much','Feeling tired or having little energy','Poor appetite or overeating','Feeling bad about yourself','Trouble concentrating on things','Moving or speaking slowly or too fast','Thoughts you would be better off dead','Difficulty these problems have caused']
p_depression_cols=['Respondent sequence number','Have little interest in doing things','Feeling down, depressed, or hopeless','Trouble sleeping or sleeping too much','Feeling tired or having little energy','Poor appetite or overeating','Feeling bad about yourself','Trouble concentrating on things','Moving or speaking slowly or too fast','Thoughts you would be better off dead','Difficulty these problems have caused']
demo_cols= ['Respondent sequence number','Data release cycle','Interview/Examination status','Gender','Age in years at screening','Age in months at screening - 0 to 24 mos','Race/Hispanic origin','Race/Hispanic origin w/ NH Asian','Six-month time period','Age in months at exam - 0 to 19 years','Served active duty in US Armed Forces','Country of birth','Length of time in US','Education level - Adults 20+','Marital status','Pregnancy status at exam','Total number of people in the Household','HH ref person’s gender','HH ref person’s age in years','HH ref person’s education level','HH ref person’s marital status','HH ref person’s spouse’s education level','Full sample 2-year interview weight','Full sample 2-year MEC exam weight','Masked variance pseudo-stratum','Masked variance pseudo-PSU','Ratio of family income to poverty']
p_demo_cols=['Respondent sequence number','Data release cycle','Interview/Examination status','Gender','Age in years at screening','Age in months at screening - 0 to 24 mos','Race/Hispanic origin','Race/Hispanic origin w/ NH Asian','Six-month time period','Country of birth','Length of time in US','Education level - Adults 20+','Marital status','Pregnancy status at exam','Language of SP Interview','Proxy used in SP Interview?','Interpreter used in SP Interview?','Language of Family Interview','Proxy used in Family Interview?','Interpreter used in Family Interview?','Language of MEC Interview','Proxy used in MEC Interview?','Interpreter used in MEC Interview?','Language of ACASI Interview','Full sample interview weight','Full sample MEC exam weight','Masked variance pseudo-PSU','Masked variance pseudo-stratum','Ratio of family income to poverty']

After loading all datasets, the next step is to match all column names to their full name to facilitate interpretation. 

In [72]:
bmx = pd.DataFrame(data = dataframes["bmx"])
p_bmx = pd.DataFrame(data = dataframes["p_bmx"])
sleep = pd.DataFrame(data = dataframes["sleep"])
p_sleep = pd.DataFrame(data = dataframes["p_sleep"])
income = pd.DataFrame(data = dataframes["income"])
p_income = pd.DataFrame(data = dataframes["p_income"])
depression = pd.DataFrame(data = dataframes["depression"])
p_depression = pd.DataFrame(data = dataframes["p_depression"])
demo = pd.DataFrame(data = dataframes["demo"])
p_demo = pd.DataFrame(data = dataframes["p_demo"])

After the above step, we have converted all items in the dictionary to data frames.

In [73]:
bmx.columns= bmx_cols
p_bmx.columns= p_bmx_cols
sleep.columns= sleep_cols
p_sleep.columns= p_sleep_cols
income.columns= income_cols
p_income.columns= p_income_cols
depression.columns= depression_cols
p_depression.columns= p_depression_cols
demo.columns= demo_cols
p_demo.columns= p_demo_cols

Now all dataframes have been matched with full columns names.

The following code shows the brief information of all 10 data frames:

In [74]:
bmx.info()
p_bmx.info()
sleep.info()
p_sleep.info()
income.info()
p_income.info()
depression.info()
p_depression.info()
demo.info()
p_demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8860 entries, 0 to 8859
Data columns (total 22 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Respondent sequence number           8860 non-null   float64
 1   Body Measures Component Status Code  8860 non-null   float64
 2   Weight (kg)                          8754 non-null   float64
 3   Weight Comment                       345 non-null    float64
 4   Recumbent Length (cm)                454 non-null    float64
 5   Recumbent Length Comment             18 non-null     float64
 6   Head Circumference (cm)              70 non-null     float64
 7   Head Circumference Comment           0 non-null      float64
 8   Standing Height (cm)                 8499 non-null   float64
 9   Standing Height Comment              134 non-null    float64
 10  Body Mass Index (kg/m**2)            8471 non-null   float64
 11  BMI Category - Children/Youth 

There are several keytakeaways from the above information of the data frames:
* All columns are presented in full name.
* All datasets contain NULL items of different features.
* Data frames of the same information group can have different number of features. For example, sleeping hours dataset before pandemic has total of 10 features while sleeping hours data collected after pandemic only has 6 features. However, the 6 features of post-pandemic dataset are included in the pre-pandemic dataset.
* All dataframes carry different features but they all have the first column as Respondent sequence number. This number is the identification of individuals within each period when the data was gathered. Therefore, it can only be matched among datasets of the same period of time. In other words, we can merge 5 pre-pandemic datasets and 5 post-pandemic datasets on the same sets of sequence number.
* Some features need further process: Average Sleeping Hours can be computed from Weekday and Weekend Sleeping hours.

## Data Processing

### BMI

From the Body Measure data sets of NHANES Examination data, subset columns: Respondent sequence number an Body Mass Index 

In [75]:
bmx = bmx[['Respondent sequence number','Body Mass Index (kg/m**2)']]
p_bmx = p_bmx[['Respondent sequence number','Body Mass Index (kg/m**2)']]

### Sleeping hours 

From the Questionaire Data's Sleep Disorders datasets, subset the sequence number and Sleep hours during workdays and weekends.

In [76]:
sleep = sleep[['Respondent sequence number','Sleep hours - weekdays or workdays','Sleep hours - weekends']]
p_sleep = p_sleep[['Respondent sequence number','Sleep hours - weekdays or workdays','Sleep hours - weekends']]

### Poverty 

From the Income dataset, subset the Respondent sequence number and poverty level index.

In [77]:
income = income[['Respondent sequence number','Family monthly poverty level index']]
p_income = p_income[['Respondent sequence number','Family monthly poverty level index']]

### Demographic

Subset Respondent sequence number, gender, age, education level and marital status from the Demographic dataset.

In [78]:
demo = demo[['Respondent sequence number','Gender','Age in years at screening','Education level - Adults 20+','Marital status']]
p_demo = p_demo[['Respondent sequence number','Gender','Age in years at screening','Education level - Adults 20+','Marital status']]

### Post-pandemic dataset

Merge all above datasets from the post-pandemic period 8/2021-8/2023.

In [79]:
post_covid = bmx.merge(sleep, on="Respondent sequence number", how='inner').merge(income, on="Respondent sequence number", how='inner').merge(depression, on="Respondent sequence number", how='inner').merge(demo, on="Respondent sequence number", how='inner')

Drop all NaN values for better analysis

In [80]:
post_covid = post_covid.dropna()

In [81]:
post_covid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3142 entries, 1 to 6336
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Respondent sequence number             3142 non-null   float64
 1   Body Mass Index (kg/m**2)              3142 non-null   float64
 2   Sleep hours - weekdays or workdays     3142 non-null   float64
 3   Sleep hours - weekends                 3142 non-null   float64
 4   Family monthly poverty level index     3142 non-null   float64
 5   Have little interest in doing things   3142 non-null   float64
 6   Feeling down, depressed, or hopeless   3142 non-null   float64
 7   Trouble sleeping or sleeping too much  3142 non-null   float64
 8   Feeling tired or having little energy  3142 non-null   float64
 9   Poor appetite or overeating            3142 non-null   float64
 10  Feeling bad about yourself             3142 non-null   float64
 11  Trouble c

From the above information of Post-Covid dataset, there are total of 3142 non-null instances, with total of 19 features.

In [82]:
post_covid.describe()

,Respondent sequence number,Body Mass Index (kg/m**2),Sleep hours - weekdays or workdays,Sleep hours - weekends,Family monthly poverty level index,Have little interest in doing things,"Feeling down, depressed, or hopeless",Trouble sleeping or sleeping too much,Feeling tired or having little energy,Poor appetite or overeating,Feeling bad about yourself,Trouble concentrating on things,Moving or speaking slowly or too fast,Thoughts you would be better off dead,Difficulty these problems have caused,Gender,Age in years at screening,Education level - Adults 20+,Marital status
count,3142.000000,3142.000000,3142.00000,3142.000000,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3142.000000,3142.000000,3142.000000,3142.000000
mean,136340.687460,30.195544,7.66359,8.209421,2.758348e+00,6.260344e-01,6.203055e-01,1.023234e+00,1.127626e+00,6.441757e-01,5.264163e-01,5.184596e-01,2.574793e-01,9.452578e-02,4.688097e-01,1.587206,52.705602,3.980267,1.789624
std,3421.196544,7.624733,1.59651,1.697583,1.620505e+00,9.521487e-01,9.045543e-01,9.978174e-01,9.295157e-01,9.070435e-01,8.371560e-01,8.498234e-01,6.916750e-01,4.517206e-01,7.396626e-01,0.492415,17.235329,1.060833,2.910308
min,130379.000000,11.100000,2.00000,2.000000,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,1.000000,20.000000,1.000000,1.000000
25%,133337.750000,24.725000,7.00000,7.000000,1.360000e+00,5.397605e-79,5.397605e-79,5.397605e-79,1.000000e+00,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,1.000000,37.000000,3.000000,1.000000
50%,136423.000000,28.800000,8.00000,8.000000,2.590000e+00,5.397605e-79,5.397605e-79,1.000000e+00,1.000000e+00,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,2.000000,56.000000,4.000000,1.000000
75%,139273.000000,34.200000,8.50000,9.000000,4.437500e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.397605e-79,5.397605e-79,1.000000e+00,2.000000,67.000000,5.000000,2.000000
max,142310.000000,69.900000,14.00000,14.000000,5.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,2.000000,80.000000,9.000000,99.000000


We can view the statistical information of the datasets using describe. It is noticable that some categories of features represent responses with no substantive answer like "Refused" or "Don't know", which explains the max values of "9" or "99" in some columns. Additionally, categorical features in the mental disorders dataset is also stored as float, not int, which needs to be transformed.

In [83]:
post_covid.loc[:,'Have little interest in doing things':'Thoughts you would be better off dead'] = post_covid.loc[:,'Have little interest in doing things':'Thoughts you would be better off dead'].astype(int)
post_covid.head()

,Respondent sequence number,Body Mass Index (kg/m**2),Sleep hours - weekdays or workdays,Sleep hours - weekends,Family monthly poverty level index,Have little interest in doing things,"Feeling down, depressed, or hopeless",Trouble sleeping or sleeping too much,Feeling tired or having little energy,Poor appetite or overeating,Feeling bad about yourself,Trouble concentrating on things,Moving or speaking slowly or too fast,Thoughts you would be better off dead,Difficulty these problems have caused,Gender,Age in years at screening,Education level - Adults 20+,Marital status
1,130379.0,33.5,9.0,9.0,5.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.397605e-79,1.0,66.0,5.0,1.0
2,130380.0,29.7,8.0,9.0,1.40,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,5.397605e-79,2.0,44.0,3.0,1.0
3,130386.0,30.2,7.5,8.0,1.45,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5.397605e-79,1.0,34.0,4.0,1.0
8,130391.0,38.9,7.5,7.5,0.94,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,1.0,2.000000e+00,2.0,33.0,3.0,3.0
9,130392.0,43.0,9.0,9.0,3.59,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.397605e-79,2.0,74.0,5.0,1.0


After tranformation, the columns representing depression symptoms are now in integer data type.

In [84]:
post_covid.loc[:,'Have little interest in doing things':'Thoughts you would be better off dead'] = post_covid.loc[:,'Have little interest in doing things':'Thoughts you would be better off dead'].replace({7: np.nan, 9.0: np.nan})

Response with values of 7 or 9 ("Refused" and "Don't know") are replaced as NaN values

In [85]:
post_covid['Depression score']=post_covid.loc[:,'Have little interest in doing things':'Thoughts you would be better off dead'].sum(axis=1,skipna=False)

Depression score is calculated by summing up severity of all depression indicators/symptoms. This depression score will later be used to categorize instances into group of depression level. 

As severity of each indicator ranks from 0 to 3 (Not at all, Several days, More than half the days, Nearly everyday) and with total of 9 indicators, depression score is at least 0 and at most 27.

In [86]:
post_covid.describe()

,Respondent sequence number,Body Mass Index (kg/m**2),Sleep hours - weekdays or workdays,Sleep hours - weekends,Family monthly poverty level index,Have little interest in doing things,"Feeling down, depressed, or hopeless",Trouble sleeping or sleeping too much,Feeling tired or having little energy,Poor appetite or overeating,Feeling bad about yourself,Trouble concentrating on things,Moving or speaking slowly or too fast,Thoughts you would be better off dead,Difficulty these problems have caused,Gender,Age in years at screening,Education level - Adults 20+,Marital status,Depression score
count,3142.000000,3142.000000,3142.00000,3142.000000,3.142000e+03,3133.000000,3135.000000,3140.000000,3139.000000,3141.000000,3140.000000,3139.000000,3137.000000,3139.000000,3.142000e+03,3142.000000,3142.000000,3142.000000,3142.000000,3118.000000
mean,136340.687460,30.195544,7.66359,8.209421,2.758348e+00,0.602617,0.602871,1.018153,1.121376,0.641515,0.521656,0.511628,0.245457,0.086015,4.688097e-01,1.587206,52.705602,3.980267,1.789624,5.347017
std,3421.196544,7.624733,1.59651,1.697583,1.620505e+00,0.846497,0.825676,0.977603,0.907225,0.894845,0.815493,0.820453,0.621931,0.358266,7.396626e-01,0.492415,17.235329,1.060833,2.910308,4.709070
min,130379.000000,11.100000,2.00000,2.000000,5.397605e-79,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.397605e-79,1.000000,20.000000,1.000000,1.000000,1.000000
25%,133337.750000,24.725000,7.00000,7.000000,1.360000e+00,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.397605e-79,1.000000,37.000000,3.000000,1.000000,2.000000
50%,136423.000000,28.800000,8.00000,8.000000,2.590000e+00,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.397605e-79,2.000000,56.000000,4.000000,1.000000,4.000000
75%,139273.000000,34.200000,8.50000,9.000000,4.437500e+00,1.000000,1.000000,1.000000,1.500000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000e+00,2.000000,67.000000,5.000000,2.000000,7.000000
max,142310.000000,69.900000,14.00000,14.000000,5.000000e+00,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,9.000000e+00,2.000000,80.000000,9.000000,99.000000,26.000000


From the above summary, all depression symptoms are now ranging from 0 to 3. Depression score range from 1 to 26, which is within the resired range.

In [87]:
post_covid['Avg sleep hours']=(post_covid['Sleep hours - weekdays or workdays']+post_covid['Sleep hours - weekends'])/2

Sleeping hours columns are aggregated into a single average sleeping hours column.

In [88]:
post_covid = post_covid.dropna()
post_covid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3118 entries, 1 to 6336
Data columns (total 21 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Respondent sequence number             3118 non-null   float64
 1   Body Mass Index (kg/m**2)              3118 non-null   float64
 2   Sleep hours - weekdays or workdays     3118 non-null   float64
 3   Sleep hours - weekends                 3118 non-null   float64
 4   Family monthly poverty level index     3118 non-null   float64
 5   Have little interest in doing things   3118 non-null   float64
 6   Feeling down, depressed, or hopeless   3118 non-null   float64
 7   Trouble sleeping or sleeping too much  3118 non-null   float64
 8   Feeling tired or having little energy  3118 non-null   float64
 9   Poor appetite or overeating            3118 non-null   float64
 10  Feeling bad about yourself             3118 non-null   float64
 11  Trouble c

Because Depression score is the variable of interest, all NaN values are again dropped from the dataset. From the information of the transformed post_covid dataset, there are now total of 3118 instances with 21 features/

In [89]:
def classify(score):
    if score < 5:
        return "1. None"
    elif 5 <= score < 10:
        return "2. Mild"
    elif 10 <= score < 15:
        return "3. Moderate"
    elif 15 <= score < 20:
        return "4. Moderately severe"
    else:
        return "5. Severe"

post_covid["Depression Level"] = post_covid["Depression score"].apply(classify)

Depression level is defined based on the Depression Score (Wang, Nan, et al., 2024): 
* None: 0–4
* Mild: 5–9
* Moderate: 10–14
* Moderately severe: 15–19
* Severe: 20–27

### Pre-pandemic dataset

The above data processing steps are applied similarly to the pre-covid dataset:

In [90]:
pre_covid = p_bmx.merge(p_sleep, on="Respondent sequence number", how='inner').merge(p_income, on="Respondent sequence number", how='inner').merge(p_depression, on="Respondent sequence number", how='inner').merge(p_demo, on="Respondent sequence number", how='inner')

Merge into one single pre-covid dataset

In [91]:
pre_covid = pre_covid.dropna()

Drop NaN values

In [92]:
pre_covid.loc[:,'Have little interest in doing things':'Thoughts you would be better off dead'] = pre_covid.loc[:,'Have little interest in doing things':'Thoughts you would be better off dead'].astype(int)

Change data type of depression dataset features into integer.

In [93]:
pre_covid.loc[:,'Have little interest in doing things':'Thoughts you would be better off dead'] = pre_covid.loc[:,'Have little interest in doing things':'Thoughts you would be better off dead'].replace({7: np.nan, 9.0: np.nan})

Transform non-responses into NaN values

In [94]:
pre_covid['Depression score']=pre_covid.loc[:,'Have little interest in doing things':'Thoughts you would be better off dead'].sum(axis=1,skipna=False)

Compute Depression score by adding values of all 9 depression indicator features

In [95]:
pre_covid['Avg sleep hours']=(pre_covid['Sleep hours - weekdays or workdays']+pre_covid['Sleep hours - weekends'])/2

Compute average sleeping hours

In [96]:
pre_covid["Depression Level"] = pre_covid["Depression score"].apply(classify)

Categorize instances into Depression levels based on their depression score

In [97]:
pre_covid = pre_covid.dropna()

Drop NaN values

In [98]:
pre_covid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4140 entries, 1 to 8963
Data columns (total 22 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Respondent sequence number             4140 non-null   float64
 1   Body Mass Index (kg/m**2)              4140 non-null   float64
 2   Sleep hours - weekdays or workdays     4140 non-null   float64
 3   Sleep hours - weekends                 4140 non-null   float64
 4   Family monthly poverty level index     4140 non-null   float64
 5   Have little interest in doing things   4140 non-null   float64
 6   Feeling down, depressed, or hopeless   4140 non-null   float64
 7   Trouble sleeping or sleeping too much  4140 non-null   float64
 8   Feeling tired or having little energy  4140 non-null   float64
 9   Poor appetite or overeating            4140 non-null   float64
 10  Feeling bad about yourself             4140 non-null   float64
 11  Trouble c

The pre-covid dataset contains total of 4140 instances and 22 features.

## Effect of Pandemic on depression level

To understand the effect of pandemic on depression level, we can compare the recorded number of people with depression before and after pandemic. However, because of different sample size between the two periods, we need to take the ratio of people with and without depression before and after pandemic.

In [99]:
pre_covid_depression = pre_covid.groupby('Depression Level')['Depression Level'].count().reset_index(name='Counts')
pre_covid_depression['Pre Covid Ratio (%)'] = pre_covid_depression['Counts']/pre_covid_depression['Counts'].sum()*100
post_covid_depression = post_covid.groupby('Depression Level')['Depression Level'].count().reset_index(name='Counts')
post_covid_depression['Post Covid Ratio (%)'] = post_covid_depression['Counts']/post_covid_depression['Counts'].sum()*100


A new dataset is created by counting number of people belonging to each depression level group. Then a ratio of each group out of total number of people is calculated. 

In [100]:
depression_compare = pre_covid_depression.merge(post_covid_depression, on='Depression Level', how='inner', suffixes=('_pre','_post')).set_index('Depression Level').rename(columns={'Counts_pre':'Pre Covid','Counts_post':'Post Covid'})

Merge into a single dataframe group by depression level groups and present the number of people as well as each group's ratio before and after the pandemic

In [101]:
depression_compare

,Pre Covid,Pre Covid Ratio (%),Post Covid,Post Covid Ratio (%)
Depression Level,,,,
1. None,2551,61.618357,1778,57.023733
2. Mild,1051,25.386473,823,26.395125
3. Moderate,358,8.647343,323,10.359205
4. Moderately severe,127,3.067633,138,4.425914
5. Severe,53,1.280193,56,1.796023


In [102]:
depression_compare_ratio = depression_compare[["Pre Covid Ratio (%)", "Post Covid Ratio (%)"]].transpose()
depression_compare_ratio.index = ["Pre-COVID", "Post-COVID"]
depression_compare_ratio = depression_compare_ratio.reset_index().rename(columns={"index": "Datasets"})
output_notebook()

Loading BokehJS ...

In the visualization, I will use stacked bar to present each group as a component of total sampled data. Therefore, the ratios are transformed as the indexes, then each group's ratio is stacked and presented in the plot.

In [103]:
depression_levels = depression_compare_ratio.columns[1:6].tolist()

p = figure(
    x_range = depression_compare_ratio["Datasets"].tolist(), 
    tooltips=[
        ("Dataset", "@Datasets"),
        ("Ratio (%)", "@$name"),
    ])

p.vbar_stack(
    depression_levels,
    x="Datasets",
    width=0.9,
    color=Spectral5[:len(depression_levels)],  
    source=depression_compare_ratio,
    legend_label=depression_levels,
)
show(p)


From the visualization, it is noticeable that there is higher proportion of people with depression after pandemic than before pandemic, especially in the depression levels from moderate to severe. From the use of bokeh library, the stacked bar chart is interactive as user can move, zoom and view the figures at different colored groups.

## Correlations of different factors with depression score

To gain more insights into how much each feature correlate with the depression score, we can merge pre-pandemic and post-pandemic datasets together and check the correlation matrix of the full dataset.

In [104]:
full_data = pd.concat([pre_covid,post_covid])
full_data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 7258 entries, 1 to 6336
Data columns (total 22 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Respondent sequence number             7258 non-null   float64
 1   Body Mass Index (kg/m**2)              7258 non-null   float64
 2   Sleep hours - weekdays or workdays     7258 non-null   float64
 3   Sleep hours - weekends                 7258 non-null   float64
 4   Family monthly poverty level index     7258 non-null   float64
 5   Have little interest in doing things   7258 non-null   float64
 6   Feeling down, depressed, or hopeless   7258 non-null   float64
 7   Trouble sleeping or sleeping too much  7258 non-null   float64
 8   Feeling tired or having little energy  7258 non-null   float64
 9   Poor appetite or overeating            7258 non-null   float64
 10  Feeling bad about yourself             7258 non-null   float64
 11  Trouble c

The full dataset contains total of 7258 instances with 22 features.

In [105]:
depression_data = full_data[['Body Mass Index (kg/m**2)','Family monthly poverty level index','Gender','Age in years at screening','Education level - Adults 20+','Marital status','Avg sleep hours','Depression score']]

To visualize the correlation among data features, a correlogram can be applied to the data set

In [106]:
pairs = list(combinations(depression_data.columns, 2))
correlations = []
for x, y in pairs:
    matrix = np.corrcoef(depression_data[x], depression_data[y])
    correlations.append(matrix[0, 1])
x, y = list(zip(*pairs))

correlation = pd.DataFrame({
    "feature_1": x,
    "feature_2": y,
    "correlation": correlations,
    "dot_size": [(1 + 10 * abs(corr)) * 10 for corr in correlations],
})

x_range = correlation["feature_1"].unique()
y_range = list(correlation["feature_2"].unique())

source = ColumnDataSource(correlation)

p = figure(x_axis_location="above",x_range=x_range,y_range=y_range,background_fill_color="#fafafa")
c = p.scatter(x="feature_1",y="feature_2",size="dot_size",source=source,fill_color=linear_cmap("correlation", "RdYlGn9", -0.5, 0.5),line_color="#202020")

color_bar = c.construct_color_bar(ticker=FixedTicker(ticks=[-0.5, 0.0, 0.5]),title="correlation")
p.add_layout(color_bar, "below")

p.xaxis.major_label_orientation = 0.8  # Rotate x-axis labels

show(p)


From the above graph, some key takeaways to be drawn are:
* Age and average sleeping hours have no correlation with depression score
* BMI, Gender and marital status have slight positive correlation with depression score
* Education level has slight negative correlation with depression score
* Of all the features, monthly poverty level index has the highest (negative) correlation with the variable of interest
* Family monthly poverty level index has high correlation with education level

### Impact of Poverty level index on Depression score

As poverty level index stood out in the correlogram, we can draw a graph of relationship between monthly poverty level index and depression score to gain more interesting insights.


In [107]:
y = full_data.loc[:,'Depression score']
x = full_data.loc[:,'Family monthly poverty level index']

source = ColumnDataSource(full_data)

p = figure(width=800, height=400,
           tools="pan,wheel_zoom", title='Depression Level by Poverty Level')
p.ygrid.grid_line_color = None
p.xaxis.axis_label = "Family Monthly Poverty Level Index"
p.yaxis.axis_label = "Depression score"

cr = p.scatter(y='Depression score',x='Family monthly poverty level index', size=10,
    fill_color="steelblue", alpha=0.1, line_color=None,
    hover_fill_color="midnightblue", hover_alpha=0.5,
    hover_line_color="white",source=source)

p.add_tools(HoverTool(tooltips=None, renderers=[cr], mode='hline'))

show(p)

Family monthly poverty level index is a ratio of monthly family income to the HHS poverty guidelines (a federal poverty threshold) specific to family size. Hence the poverty level index represents a family's income over a poverty threshold. Higher monthly poverty level index indicates higher monthly income and vice versa.

According to the above visualization, most data points are within low depression score, from 0-5 and they are spread out in all ranges of poverty level index. However, when we move up to higher depression score, it is noticeable that the data points belong to low poverty level index values (from 0 to 2). This shows that families with lower income and are closer to the poverty threshold are more likely to have depression than those with decent income.

### Distributions of people with depression over age groups and genders

To understand more about the distribution of people with depression from moderate level and above (depression score of at least 10) over different age groups and genders, we first subset age and gender column and add an condition that depression score is higher than or equal to 10.

In [108]:
depression_by_age_gender = full_data[full_data["Depression score"]>=10][["Age in years at screening", "Gender"]]
depression_by_age_gender.rename(columns={"Age in years at screening": "age", "Gender": "gender"}, inplace=True)
depression_by_age_gender.head()

,age,gender
2,36.0,1.0
39,43.0,2.0
40,23.0,2.0
61,29.0,2.0
83,59.0,2.0


From the NHANES dataset, value of gender column represents male as 1 and female as 2. Therefore, we can transform these numerical values into categorical "male" and "female".

In [109]:
depression_by_age_gender["gender"] = depression_by_age_gender["gender"].map({1: "male", 2: "female"})

To plot a pyramid, two separate data frames of male and female distribution in age groups are created as 2 histograms of the pyramid.

In [110]:
male_ages = depression_by_age_gender[depression_by_age_gender["gender"] == "male"]["age"]
female_ages = depression_by_age_gender[depression_by_age_gender["gender"] == "female"]["age"]

In [126]:
bin_width = 5
bins = np.arange(0, depression_by_age_gender["age"].max() + bin_width, bin_width)
m_hist, edges = np.histogram(male_ages, bins=bins)
f_hist, edges = np.histogram(female_ages, bins=bins)

age_groups = [f"{int(edges[i])}-{int(edges[i+1])}" for i in range(len(edges) - 1)]

p = figure(height=600,width=800,x_range=(-80, 80),y_range=list(age_groups),x_axis_label="Count",y_axis_label="Age Groups")

p.hbar(right=m_hist*-1,y=age_groups, height=0.8, color=colors[1], line_width=0,  legend_label="Male")
p.hbar(right=f_hist, y=age_groups, height=0.8, color=colors[0], line_width=0,  legend_label="Female")

p.legend.location = "top_left"

show(p)

Although from the pyramid, it looks like women are more likely to have depression than men. However, this is mostly affected by the fact that sampled group of people are mostly female. Therefore the analysis will focus on the distribution of depressed adult in different age groups between male and female.

From the pyramid, there is a similar trend for both male and female sampled individuals that most people with depression are in the age of 60-65. For women, the highest number of depressed people were seen from 55 to 65. There is also a significant number of men and women with depression in the young age groups of 20 to 35.

### Distributions of people with depression over marital status and education level

Now we will dive into the dataset of people with depression in terms of their education level and marital status.

In [135]:
depression_by_mar_edu = full_data[full_data["Depression score"]>=10][["Education level - Adults 20+", "Marital status"]]

From the above code, we have created a new dataframe by subsetting education level and marital status

In [136]:
depression_by_mar_edu = depression_by_mar_edu[depression_by_mar_edu["Marital status"].isin([1, 2, 3]) & depression_by_mar_edu["Education level - Adults 20+"].isin([1, 2, 3, 4, 5])]
depression_by_mar_edu["Marital status"] = depression_by_mar_edu["Marital status"].map({1: "Married", 2: "Widowed/Divorced/Separated", 3:"Never married"})
depression_by_mar_edu

,Education level - Adults 20+,Marital status
2,4.0,Never married
39,4.0,Widowed/Divorced/Separated
40,3.0,Never married
61,4.0,Never married
83,4.0,Widowed/Divorced/Separated
...,...,...
6298,3.0,Widowed/Divorced/Separated
6302,3.0,Widowed/Divorced/Separated
6305,5.0,Widowed/Divorced/Separated
6316,4.0,Never married


Similarly to the non-responses values of the depression dataset, we also need to drop the "Refused" and "Don't know" values of the marital status and education level features. After that, numerical values of marital status feature is then mapped with their full categorical values.

In [157]:
mar_edu_count = depression_by_mar_edu.groupby(["Education level - Adults 20+", "Marital status"]).size().reset_index(name="Count")
mar_edu_count = mar_edu_count.pivot(index="Education level - Adults 20+", columns="Marital status", values="Count")
mar_edu_count.index.name = "Education Level"
mar_edu_count

Marital status,Married,Never married,Widowed/Divorced/Separated
Education Level,,,
1.0,30,12,22
2.0,51,32,37
3.0,113,69,86
4.0,159,101,139
5.0,85,70,47


In the above code, a new dataframe is created to count the number of people with depression of each education level and marital status. Then the dataset is transformed into a table with each marital status as a single column for visualization. The index name is changed to "Education Level" for presentation.

In [158]:
source = ColumnDataSource(mar_edu_count)

p = figure(x_axis_label="Education Level",y_axis_label="Number of People",height=400,width=700)
colors = Spectral4

for i, status in enumerate(data.columns):
    p.line(x="Education Level",y=status,source=source,line_width=3,color=colors[i],legend_label=status)

p.legend.location = "top_left"
p.legend.click_policy="hide"

show(p)


The bokeh library supports user interation in hiding glyphs when some particular data group is more focused on in the analysis.

From the bokeh graph, we can see that most sampled individuals are from education level 3 to 5 (highschool graduate, college graduate and above). This is true to people of all marital status groups. However, it is noticeable that although the majority of the dataset are individuals that never married, the highest number of people with depression is seen in the Married group, and singles (never married) see the lowest number of people with depression. This is true in all level of education from 1 to 4. However, for those in education level 5 (college graduate or above), there are more depressed single people that never married, than those who are widowed, divorced or separated.

### Data privacy and ethics

From the above analysis of factors concerning mental disorders of NHANES dataset, it is a good example of how multidimensional data is accessible publicly and how it can be tranformed to create meaningful insights. Although personal identifiers (ID number, Name, Adress, Phone number...) have been removed, there is still risk of being reidentified using various demographic data, or laboratory data, given the access to other external data sources. The lack of anonymity can lead to privacy issues, especially for sensitive topics like mental health. 

There are also several ethical issues worth mentioning. Participants may not understand fully how their data is being used or will be published, which can pose harm to their confidentiality and may damage public trust in research. Bias in data collection and analysis can also lead to misleading conclusions, which may result stereotyping or discrimination. Finally, these findings can also be misused for commercial or political purposes, which could undermine the intent of research aim at improving public well-being. 

## Conclusion

This analysis focus on the prevalence and distribution of depression in the United States across demographic factors, including age, gender, income, education level, and marital status. By leveraging pre-pandemic (2017–March 2020) and post-pandemic (August 2021–August 2023) data from NHANES, the report provides valuable insights into how depression trends have been changed before and after the pandemic.

Key findings reveal that middle-aged individuals, both men and women, face a higher burden of depression, while the impact of socio-economic factors such as income, education and marital status also play a significant role to mental health disorders. Understanding these patterns is critical for designing targeted mental health interventions and policies that address the needs of vulnerable groups. By acknowledging the role of social and economic factors and the long-term effects of the COVID-19 pandemic, this report emphasizes the importance of comprehensive mental health strategies to mitigate the growing burden of depression and improve overall well-being in the US population.

## References

1. Cdc.gov. (2010). NHANES Questionnaires, Datasets, and Related Documentation. [online] Available at: https://wwwn.cdc.gov/nchs/nhanes/Default.aspx.

2. Wang, N., Yan, X., Imm, K., Xu, T., Li, S., Gawronska, J., Wang, R., Smith, L., Yang, L. and Cao, C. (2024). Racial and ethnic disparities in prevalence and correlates of depressive symptoms and suicidal ideation among adults in the United States, 2017–2020 pre-pandemic. Journal of Affective Disorders, 345, pp.272–283. doi:https://doi.org/10.1016/j.jad.2023.10.138.

2. Bokeh. (2015). stacked. [online] Available at: https://docs.bokeh.org/en/latest/docs/examples/basic/bars/stacked.html

3. Bokeh. (2024). correlogram. [online] Available at: https://docs.bokeh.org/en/latest/docs/examples/topics/categorical/correlogram.html

4. Bokeh. (2024). glyph_hover. [online] Available at: https://docs.bokeh.org/en/latest/docs/examples/styling/plots/glyph_hover.html

5. Bokeh. (2024). pyramid. [online] Available at: https://docs.bokeh.org/en/latest/docs/examples/topics/stats/pyramid.html

6. Bokeh. (2024). legend_hide. [online] Available at: https://docs.bokeh.org/en/latest/docs/examples/interaction/legends/legend_hide.html